In [ ]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)
 
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme(palette='pastel')
plt.rcParams['figure.figsize'] = (12, 6)
pd.set_option('display.max_columns', None)

In [ ]:
# Carga la base de TrackCare
df_track = pd.read_csv("../data/processed/trackcare_procesada.csv")

# Deja solamente las consultas atendidas
df_track_atendidas = df_track.query("estadocita == 'Atendido'")

In [ ]:
def obtener_distribucion_consultas(df, agrupacion):
    # Agrupa segun lo especificado por el usuario y separa por pacientes
    agrupacion_por_paciente = agrupacion + ["id_paciente"]

    # Obtiene la cantidad de consultas por paciente
    consultas_por_paciente = (
        df.groupby(agrupacion_por_paciente).size().reset_index(name="n_consultas")
    )

    # Obtiene el desempeno de las consultas
    distribucion_consultas = consultas_por_paciente.groupby(agrupacion)["n_consultas"].describe()

    # Obtiene la cantidad de consultas totales por especialidad
    cantidad_consultas = df.groupby(agrupacion).size()
    cantidad_consultas.name = "cantidad_consultas"

    # Agrega la cantidad de consultas por especialidad
    distribucion_consultas = distribucion_consultas.join(cantidad_consultas)

    return distribucion_consultas, consultas_por_paciente

In [ ]:
# Obtiene resumen de consultas por anio
agrupacion_anual = ["ano", "especialidad"]
distribucion_consultas_por_anio, consultas_por_paciente_por_anio = obtener_distribucion_consultas(
    df_track_atendidas, agrupacion_anual
)

# Obtiene resumen de consultas acumuladas en el periodo
agrupacion_acumulada = ["especialidad"]
distribucion_consultas_acumuladas, consultas_por_paciente_acumuladas = (
    obtener_distribucion_consultas(df_track_atendidas, agrupacion_acumulada)
)

In [ ]:
# Exporta la cantidad de consultas por paciente, eliminando su ID
consultas_por_paciente.drop(columns="id_paciente").to_csv(
    "../data/interim/distribucion_consultas.csv", index=False
)